In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
data_folder = os.getcwd()+'\\IO'
gams_folder = os.getcwd()+'\\Models'

# *Condition trees used to define groups of variables*

*Groups of models are defined using dictionaries of the gpy_symbols that are conditioned using the condition trees (see ConditionTrees.ipynb).*

### 0: Initialize and ready a model instance

*Load a model instance of a production sector from a pickle:*

In [2]:
gm = Production.pr_dynamic(pickle_path=gams_folder+'\\gmspython_p')

*The definition of groupings are defined in the 'gams\_settings' class, stored under model.settings:*

In [3]:
gm.model.settings.__dict__

{'name': 'p',
 'placeholders': {'p_1': 'p_1'},
 'setstate': 'B',
 'solvestat': True,
 'files': {'p_functions.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\GPM_v05\\examples\\gamsmodels\\ProductionDynamic',
  'p_sets.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\GPM_v05\\examples\\gamsmodels\\ProductionDynamic',
  'p_parameters.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\GPM_v05\\examples\\gamsmodels\\ProductionDynamic',
  'p_groups.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\GPM_v05\\examples\\gamsmodels\\ProductionDynamic',
  'p_blocks.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\GPM_v05\\examples\\gamsmodels\\ProductionDynamic',
  'p_CollectFile.gms': 'C:\\Users\\sxj477\\Documents\\GitHub\\GPM_v05\\examples\\gamsmodels\\ProductionDynamic'},
 'collect_files': <DB2Gams_l2.OrdSet at 0x1f6bdd48148>,
 'root_file': None,
 'data_folder': 'C:\\Users\\sxj477\\Documents\\GitHub\\GPM_v05\\examples\\gamsmodels\\ProductionDynamic',
 'import_settings': {},
 'export_settings': {'dropattrs': ['ws', 

*In particular, the definition of groups are stored in the configuration attribute ('conf'), under the keywords 'g\_exo' and 'g\_endo':*

In [4]:
display('Ordered set with exogenous groups:', gm.model.settings.conf['B']['g_exo'].v, 'Ordered set with endogenous groups:', gm.model.settings.conf['B']['g_endo'].v)

'Ordered set with exogenous groups:'

['p_g_tech', 'p_g_vars_exo', 'p_ict_exo']

'Ordered set with endogenous groups:'

['p_g_vars_endo', 'p_ict_endo']

### 1: Allowed specifications for grouping of variables

Groups can be specified in three ways:
* As a dictionary where keys = variable names, values = conditions on the variable.
* As a string, referring to the name of another group.
* As a list with ['-',X], where X is either a dictionary/string. This operation removes elements from the group (the elements can be variables or groups). 

The groups should be specified using the *self.group_conditions(self,group)* method, where a group can be specified using a combination of three methods described above. Here we define some examples of syntax that can be applied:

In [5]:
def group_conditions(group):
    if group =='g1':
        return [{'sigma': None, 'mu': gm.g('exo_mu')}]
    elif group == 'g2':
        return ['g_tech_exo',{'Peq': gm.g('n_out')}]
    elif group == 'g3':
        return ['g_tech_exo',['-',{'sigma': gm.g('kno_inp')}]]

* 'g1': Specifies a group consisting of variables. The variable 'sigma' is not sliced (i.e. no conditions are applied), whereas only a subset of the values of the variable 'mu' are used. The conditions to subset the variable used in the definition here follows that of condition trees (see ConditionTrees.ipynb for more).
* 'g2': A group that consists of the group 'g_tech_exo', as well as the variable 'Peq' (sliced according to the gpy\_symbol 'n_out').
* 'g3': A group that consists of the group 'g_tech_exo', but without the values in 'sigma' defined over the domain 'kno_inp'.

*Note: We refer to the group g_tech_exo, as this is a defined name in the gmspython model instance we are working on here (the 'gm' object)*

### 2: Getting a Python representation of the groups

The *gmspython* class defines methods for looking up the variables defined the groupings. This uses a combination of simple functions to combine and merge conditions, but ultimately always draws on the condition trees, we referred to earlier.

The function gmspython.gms_aux.ReadCondition(g,model) interprets the group conditions as a single condition tree. Here *g* is a list of conditions (as defined in the group_conditions function above), and *model* is a gmspython model instance with the relevant variables defined in the database:

*Condition tree for group:*

In [6]:
g = 'g1'
gmspython.gms_aux.ReadCondition(group_conditions(g),gm)

{'mu': <DataBase.gpy_symbol at 0x1f6bdd24cc8>, 'sigma': None}

*variables subsetted with the relevant condition (using the rctree_pd method, see ConditionTrees.ipynb for more):*

In [7]:
var = 'sigma'
gm.g(var).rctree_pd(gmspython.gms_aux.ReadCondition(group_conditions(g),gm)[var])

s  n   
a  K       0.25
   M       0.25
   a       0.25
   aa_F    2.00
   bb_F    2.00
b  K       0.20
   M       0.30
   aa_F    1.50
   b       0.60
   bb_F    1.50
Name: sigma, dtype: float64

The method illustrated here, to get the Python representation of a variable that is subsetted using the group definitions are also implemented directly in the *gmspython* class; The method *var_custom_group(self,symbol,group,db=None)* uses custom group definitions as ('g1','g2','g3') defined above to subset a variable (symbol):

In [8]:
gm.var_custom_group(var,group_conditions(g))

s  n   
a  K       0.25
   M       0.25
   a       0.25
   aa_F    2.00
   bb_F    2.00
b  K       0.20
   M       0.30
   aa_F    1.50
   b       0.60
   bb_F    1.50
Name: sigma, dtype: float64

Four other functions (that relies on the *var_custom_group* method) are similarly implemented: 
* *self.var_ss(self,symbol,group,db=None):* Returns the sliced variable 'symbol' from the group 'group' specified as a string. If db=None the lookup for symbols occur through the *self.model.database* method. 
* *self.var_exo(self,symbol,db=None):* Returns the sliced variable 'symbol', using the union of all exogenous groups. 
* *self.var_endo(self,symbol,db=None):* Similar to var_exo, but with endogenous values.
* *self.slice_exo(db,copy=True,copy_kwargs={}):* Returns a representation of the database 'db' that only includes variables that are exogenous in the *self* model. If copy = True the function returns a copy of the database 'db'; otherwise the slicing occurs inplace. copy_kwargs are parsed to the initialization of the new database (thus can include name, workspace etc.).

*Example of the four methods:*

*Subset of the variable 'sigma' in the group 'g_tech_exo':*

In [9]:
gm.var_ss('sigma','g_tech_exo')

s  n   
a  K       0.25
   M       0.25
   a       0.25
   aa_F    2.00
   bb_F    2.00
b  K       0.20
   M       0.30
   aa_F    1.50
   b       0.60
   bb_F    1.50
Name: sigma, dtype: float64

*Subset of the variable 'sigma' that is exogenous in the model:*

In [10]:
gm.var_exo('sigma')

s  n   
a  K       0.25
   M       0.25
   a       0.25
   aa_F    2.00
   bb_F    2.00
b  K       0.20
   M       0.30
   aa_F    1.50
   b       0.60
   bb_F    1.50
Name: sigma, dtype: float64

*Subet of the variables 'sigma' and 'qD' that are endogenous:*

In [11]:
gm.var_endo('sigma'),gm.var_endo('qD')

(None,
 t  s  n   
 1  a  I_iB     0.998536
       I_iM     1.217273
       K        1.963273
       L        3.803379
       M       12.839989
       a        3.530603
       a_F      2.516185
       aa_F     5.815327
       b        2.486219
       b_F      0.484490
       bb_F     7.394292
    b  I_iB     0.998536
       I_iM     1.217273
       K        1.563906
       L        7.028736
       M       44.248796
       a        1.372055
       a_F      0.603783
       aa_F    49.558400
       b       20.955930
       b_F      3.175601
       bb_F    17.006030
 2  a  I_iB     0.998536
       I_iM     1.217273
       K        1.963273
       L        3.803379
       M       12.839989
       a        3.530603
       a_F      2.516185
       aa_F     5.815327
       b        2.486219
       b_F      0.484490
       bb_F     7.394292
       iB      16.000000
       iM       8.000000
    b  I_iB     0.998536
       I_iM     1.217273
       K        1.563906
       L        7.028736
      

### 3: Getting at the gams/gamY code written from the groups

The same group conditions used to subset variables in a Python representation can be used to write gams code: (1) Dictionaries defining variables, (2) strings adding entire groups, (3) lists subtracting specific elements (variables and/or groups). The main method for writing gams code is the *self.add_group* method defined in the *gmspython* class. Relying on the pre-programmed groupings defined in the *Production.py* model, we can see how the add_group returns a formatted version of the conditions:

In [12]:
gm.add_group('g_tech_exo')

[{'name': 'sigma', 'conditions': <DataBase.gpy_symbol at 0x1f6bdd24048>},
 {'name': 'eta', 'conditions': <DataBase.gpy_symbol at 0x1f6bdd1ffc8>},
 {'name': 'mu', 'conditions': <DataBase.gpy_symbol at 0x1f6bdd24cc8>}]

When executing the main write and run methods for the *gmspython* class, these groups are interpreted by the *DB2Gams.gams_model_py* class; in particular by the *add_group_to_groups* method. The *add_group_to_groups* method uses the list of elements (e.g. dictionaries) and adds it to the *self.model.groups* attribute. This is initially empty:

In [14]:
gm.model.groups

{}

Writing the 'g_tech_exo' group means running:

In [16]:
g = 'g_tech_exo'
gm.model.add_group_to_groups(gm.add_group(g),g)

The groups now has a group called 'g_tech_exo' with 'sigma','eta', and 'mu':

In [17]:
gm.model.groups

{'g_tech_exo': [{'name': 'sigma',
   'conditions': <DataBase.gpy_symbol at 0x1f6bdd24048>},
  {'name': 'eta', 'conditions': <DataBase.gpy_symbol at 0x1f6bdd1ffc8>},
  {'name': 'mu', 'conditions': <DataBase.gpy_symbol at 0x1f6bdd24cc8>}]}

When the *self.model* is compiled, the *self.model.groups* are written to gams code using the *DB2Gams* method *add_group:*

In [19]:
print(DB2Gams.add_group(gm.model.groups['g_tech_exo'],gm.model.database))

sigma[s,n]$(kno_inp[s,n]) ""
eta[s,n]$(kno_out[s,n]) ""
mu[s,n,nn]$(exo_mu[s,n,nn]) ""



The *self.model* executes this in the *self.model.write_group* method. Thus, the complete block of the group is written as:

In [21]:
print(gm.model.write_group(g))

$GROUP g_tech_exo
sigma[s,n]$(kno_inp[s,n]) ""
eta[s,n]$(kno_out[s,n]) ""
mu[s,n,nn]$(exo_mu[s,n,nn]) ""
;


